<a href="https://colab.research.google.com/github/SaKi1309/ML_Notebooks/blob/main/GAN_distributed_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, TensorFlow's tutorial on the [DCGAN](https://www.tensorflow.org/tutorials/generative/dcgan) is adapted to be able to train the model on TPU. 
[Check this guide]() for details on distributed training on TPU. 

<big> **Don't forget to turn on TPU in Runtime->Change runtime type->TPU!** 💪</big>

## Notebook Settings
The following Parameters can be adapted to influence the behavior of the notebook.

In [ ]:
IMAGE_SHAPE = (28,28,1) #@param
NOISE_DIM = 100 #@param {type:"integer"} 
GLOBAL_BATCH_SIZE = 256 #@param {type:"integer"}
EPOCHS =  30 #@param {type:"integer"}
NUM_EXAMPLES_TO_GENERATE = 16 #@param {type:"integer"}

<a name="0"></a>
# 0 - Initial Steps

In [ ]:
!pip install imageio
!pip install git+https://github.com/tensorflow/docs
!pip install DeepSaki

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time
import random

from IPython import display
import DeepSaki

tf.__version__
#DeepSaki.utils.EnableXlaAcceleration()

In [ ]:
strategy, RUNTIME_ENVIRONMENT, hw_accelerator_handle = DeepSaki.utils.DetectHw()

In [ ]:

CONFIG = {
    "BUFFER_SIZE":60000,
    "BATCH_SIZE_PER_REPLICA":int(GLOBAL_BATCH_SIZE / strategy.num_replicas_in_sync),
    "GLOBAL_BATCH_SIZE":GLOBAL_BATCH_SIZE,
    "REDUCTION_STRATEGY":tf.keras.losses.Reduction.NONE,
    "EPOCHS":EPOCHS,
    "NOISE_DIM":NOISE_DIM,
    "NUM_EXAMPLES_TO_GENERATE":NUM_EXAMPLES_TO_GENERATE,
    "NOISE_SHAPE":(NOISE_DIM,),
    #"IMAGE_SHAPE":(32,32,3)
    "IMAGE_SHAPE": IMAGE_SHAPE
}

GLOBAL_PARAM = {
    "RANDOM_SEED": 10
}

In [ ]:
#Seed is used to have comparability between runs while maintaining random behavior!
random.seed(GLOBAL_PARAM["RANDOM_SEED"])
np.random.seed(seed=GLOBAL_PARAM["RANDOM_SEED"])

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
latent_noise = tf.random.normal([CONFIG["NUM_EXAMPLES_TO_GENERATE"], CONFIG["NOISE_DIM"]])

<a name="1"></a>
# 1 - Dataset

The dataset is loaded, pre-processed and finally, a distributed dataset is created.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], *CONFIG["IMAGE_SHAPE"]).astype('float32')
test_images = test_images.reshape(test_images.shape[0], *CONFIG["IMAGE_SHAPE"]).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
test_images = (test_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

print("Shape of train_images: {}".format(np.shape(train_images)))
print("Shape of test_images: {}".format(np.shape(test_images)))

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(CONFIG["BUFFER_SIZE"]).batch(CONFIG["GLOBAL_BATCH_SIZE"],drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices(test_images).shuffle(CONFIG["BUFFER_SIZE"]).batch(CONFIG["GLOBAL_BATCH_SIZE"],drop_remainder=True)

#distribute
train_dataset_distributed = strategy.experimental_distribute_dataset(train_dataset)
test_dataset_distributed = strategy.experimental_distribute_dataset(test_dataset)

<a name="2"></a>
# 2 - Model Definition

<a name="2-1"></a>
## 2.1. Generator

In [ ]:
def Generator(inputShape = CONFIG["NOISE_SHAPE"]):
  # specify the input shape
  input_tensor = tf.keras.layers.Input(shape = inputShape)
 
  x = tf.keras.layers.Dense(7*7*256, use_bias=False)(input_tensor)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU()(x)

  x = tf.keras.layers.Reshape((7, 7, 256))(x)

  x = tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU()(x)

  x = tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU()(x)

  x = tf.keras.layers.Conv2DTranspose(CONFIG["IMAGE_SHAPE"][-1], (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)

  # create the model
  model = tf.keras.Model(inputs=input_tensor, outputs=x, name = "Generator")

  return model

#Testcode
model = Generator()
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=False, show_dtype=True, to_file="generator.png")

Important to note, that the reduction strategy is set to tf.keras.losses.Reduction.NONE. Additionall,in line 4, the loss is reduced by summing all values and is scaled by the global batch size, not the per_replica_batch_size.

In [ ]:
def generator_loss(fake_output):
  cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True,reduction=CONFIG["REDUCTION_STRATEGY"])
  loss = cross_entropy(tf.ones_like(fake_output), fake_output)
  loss = tf.reduce_sum(loss) *(1. / CONFIG["GLOBAL_BATCH_SIZE"])
  return loss

<a name="2-2"></a>
## 2.2. Discriminator

In [ ]:
def Discriminator(inputShape = CONFIG["IMAGE_SHAPE"]):
  # specify the input shape
  input_tensor = tf.keras.layers.Input(shape = inputShape)
  x = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(input_tensor)
  x = tf.keras.layers.LeakyReLU()(x)
  x = tf.keras.layers.Dropout(0.3)(x)

  x = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
  x = tf.keras.layers.LeakyReLU()(x)
  x = tf.keras.layers.Dropout(0.3)(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1)(x)

  # create the model
  model = tf.keras.Model(inputs=input_tensor, outputs=x, name = "Discriminator")

  return model

#Testcode
model = Discriminator()
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=False, show_dtype=True, to_file="discriminator.png")

As for the generator loss, the discriminator loss sets the reduction strategy to NONE, reduces by summing all values and scales by the global batch size

In [ ]:
def discriminator_loss(real_output, fake_output):
  cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True,reduction=CONFIG["REDUCTION_STRATEGY"])
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  total_loss = tf.reduce_sum(total_loss) *(1. / CONFIG["GLOBAL_BATCH_SIZE"])
  return total_loss

<a name="2-3"></a>
## 2.3. GAN

In [ ]:
class GAN(tf.keras.Model):
  def __init__(self, generator, discriminator, strategy = None):
    super(GAN, self).__init__()

    self.generator = generator
    self.discriminator = discriminator
    self.strategy = strategy

  def compile(self, generator_optimizer, discriminator_optimizer, generator_loss, discriminator_loss):
    super(GAN, self).compile()
    #super(VoloGAN, self).compile(steps_per_execution = 64)
    self.generator_optimizer = generator_optimizer
    self.discriminator_optimizer = discriminator_optimizer
    self.generator_loss = generator_loss
    self.discriminator_loss = discriminator_loss

  def call(self, batch):
    generated = self.generator(batch)
    discriminated = self.discriminator(generated)
    return tf.stack([generated, discriminated])

  def print_summary(self):
    def getNumberOfElements(listOfLayers):
      numElements = 0
      for layer in listOfLayers:
        numElements += tf.reduce_prod(layer.shape)
      return numElements

    summaryString = ""
    summaryString += "--------------------------------------------------\n"
    summaryString += "------------------- Summary GAN ------------------\n"
    summaryString += "--------------------------------------------------\n"
    
    summaryString += "\n"
    summaryString += self.generator.name + ":\n"
    gen_total = getNumberOfElements(self.generator.variables)
    gen_trainable = getNumberOfElements(self.generator.trainable_variables)
    gen_nonTrainable = gen_total - gen_trainable
    summaryString += "  Total Variables: {:,}\n".format(gen_total)
    summaryString += "  Trainable Variables: {:,}\n".format(gen_trainable)
    summaryString += "  Non-trainable Variables: {:,}\n".format(gen_nonTrainable)

    summaryString += "\n"
    summaryString += self.discriminator.name + ":\n"
    disc_total = getNumberOfElements(self.discriminator.variables)
    disc_trainable = getNumberOfElements(self.discriminator.trainable_variables)
    disc_nonTrainable = disc_total - disc_trainable
    summaryString += "  Total Variables: {:,}\n".format(disc_total)
    summaryString += "  Trainable Variables: {:,}\n".format(disc_trainable)
    summaryString += "  Non-trainable Variables: {:,}\n".format(disc_nonTrainable)

    summaryString += "_________________________________________________\n"

    GAN_total = gen_total + disc_total
    GAN_trainable = gen_trainable + disc_trainable
    GAN_nonTrainable = gen_nonTrainable + disc_nonTrainable
    summaryString += "Total Variables: {:,}\n".format(GAN_total)
    summaryString += "Trainable Variables: {:,}\n".format(GAN_trainable)
    summaryString += "Non-trainable Variables: {:,}\n".format(GAN_nonTrainable)

    summaryString += "_________________________________________________\n"

    print(summaryString)

    return summaryString

  def generate_and_save_images(self, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = self.generator(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))
    fig.suptitle("Epoch: {}".format(epoch), fontsize=10)

    for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      if CONFIG["IMAGE_SHAPE"][-1]==1:
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      else:
        plt.imshow(predictions[i, :, :, 0:3] * 0.5 + 0.5)
      plt.axis('off')

    with tf.device('/job:localhost'): 
      plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

  @tf.function
  def train_step(self, batch):
    noise = tf.random.normal([CONFIG["BATCH_SIZE_PER_REPLICA"], CONFIG["NOISE_DIM"]])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = self.generator(noise, training=True)

      real_output = self.discriminator(batch, training=True)
      fake_output = self.discriminator(generated_images, training=True)

      gen_loss = self.generator_loss(fake_output)
      disc_loss = self.discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

    self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
    self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
    return [gen_loss, disc_loss]

  @tf.function
  def test_step(self, batch):
    noise = tf.random.normal([CONFIG["BATCH_SIZE_PER_REPLICA"], CONFIG["NOISE_DIM"]])
    generated_images = self.generator(noise, training=False)

    real_output = self.discriminator(batch, training=False)
    fake_output = self.discriminator(generated_images, training=False)

    gen_loss = self.generator_loss(fake_output)
    disc_loss = self.discriminator_loss(real_output, fake_output)

    return [gen_loss, disc_loss]

  @tf.function
  def distributed_train_step(self, batch_train):
    ''' 
    per_replica_loss_vector:  vector of shape [per_replica-loss_gen1, per_replica-loss_gen2, per_replica-loss_disc1, per_replica-loss_disc1]
    reduced_loss_vector:      Vector of results of the different devices (for TPU 8)
    
    '''
    per_replica_loss_vector = self.strategy.run(self.train_step, args=(batch_train,))

    #reduce the result of the replicas for every loss value returned!
    reduced_loss_vector = []
    for per_replica_loss in per_replica_loss_vector:
      reduced_loss_vector.append(self.strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None))
    
    return reduced_loss_vector

  @tf.function
  def distributed_test_step(self, batch_test):
    ''' 
    per_replica_loss_vector:  vector of shape [per_replica-loss_gen1, per_replica-loss_gen2, per_replica-loss_disc1, per_replica-loss_disc1]
    reduced_loss_vector:      Vector of results of the different devices (for TPU 8)
    
    '''
    per_replica_loss_vector = self.strategy.run(self.test_step, args=(batch_test,))

    #reduce the result of the replicas for every loss value returned!
    reduced_loss_vector = []
    for per_replica_loss in per_replica_loss_vector:
      reduced_loss_vector.append(self.strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None))
    
    return reduced_loss_vector

  def train(self, train_dataset_distributed, test_dataset_distributed, epochs):
    start_training = time.time()
    for epoch in range(epochs):
      start_epoch = time.time()

      # initialize training parameter
      total_train_loss = [0., 0.]
      num_train_batches = 0

      for image_batch in train_dataset_distributed:
        total_train_loss = tf.math.add(total_train_loss, self.distributed_train_step(image_batch))
        num_train_batches += 1
      total_train_loss = total_train_loss / float(num_train_batches)

      if test_dataset_distributed is not None:
        # initialize training parameter
        total_test_loss = [0., 0.]
        num_test_batches = 0

        for image_batch in test_dataset_distributed:
          total_test_loss = tf.math.add(total_test_loss, self.distributed_test_step(image_batch))
          num_test_batches += 1
        total_test_loss = total_test_loss / float(num_test_batches)

      # Produce images for the GIF as you go
      display.clear_output(wait=True)
      self.generate_and_save_images(epoch + 1, latent_noise)

      print ('Total training time is {0:.3f} sec'.format(time.time()-start_training))
      print ('Time for epoch {0:5} is {1:.3f} sec'.format(epoch + 1, time.time()-start_epoch))
      print("Train:\nGen. Loss: {0:.6f} \nDisc. Loss: {1:.6f}".format(total_train_loss[0], total_train_loss[1]))
      if test_dataset_distributed is not None:
        print("Test:\nGen. Loss: {0:.6f} \nDisc. Loss: {1:.6f}".format(total_test_loss[0], total_test_loss[1]))

    # Generate after the final epoch
    display.clear_output(wait=True)
    self.generate_and_save_images(epochs, latent_noise)
    print ('Total training time is {0:.3f} sec'.format(time.time()-start_training))


<a name="3"></a>
# 3 - Training

It'S important to note, that the model is initialized within the context of `strategy.scope()`

In [ ]:
with strategy.scope():
  ganModel = GAN(
      Generator(),
      Discriminator(),
      strategy
      )
ganModel.compile(
    generator_optimizer = tf.keras.optimizers.Adam(1e-4),
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4),
    generator_loss = generator_loss, 
    discriminator_loss = discriminator_loss
)
ganModel.print_summary()



In [ ]:
ganModel.train(
    train_dataset_distributed=train_dataset_distributed,
    test_dataset_distributed = None,# test_dataset, 
    epochs = CONFIG["EPOCHS"])

<a name="4"></a>
# 4 - Visualization

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(CONFIG["EPOCHS"])

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)